## Requirments

In [35]:
from dataclasses import dataclass
from pydantic import BaseModel, Field, ValidationError, field_validator
import sys

## Introduction

`pydantic` is a Python library to represent data.  It is similar to Python's `dataclass`, but with more features, especially for data validation. `pydantic` relies heavily on type annotations.

## Simple example

Defining a class is as simple as listing the object's attributes, similar to Python's own `dataclass`.  However, rather than using a decorator, a `pydantic` class is derived from `BaseModel`.

In [2]:
class Person(BaseModel):
    id: int
    age: int
    name: str

Creating objects of type `Person` is done the usual way.

In [3]:
p1 = Person(id=1, age=72, name='Maria')

In [4]:
p1

Person(id=1, age=72, name='Maria')

However, when you try do do so using values of an inapropriate type, a `ValidationError` will be raised.

In [5]:
try:
    _ = Person(id=1, age='old', name='Maria')
except ValidationError as e:
    print(e, file=sys.stderr)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='old', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing


In the example above, the string `'old'` can not be converted to an integer.  The example below is more interesting, since the integer `12` could be converted to a string for the `name` attribute.

In [6]:
try:
    _ = Person(id=1, age=72, name=12)
except ValidationError as e:
    print(e, file=sys.stderr)

1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type


It is interesing to compare this with Python's `dataclass`.

In [7]:
@dataclass
class PersonDC:
    id: int
    age: int
    name: str

In [8]:
p1_dc = PersonDC(id=3, age=49, name='Betty')

In [9]:
p1_dc

PersonDC(id=3, age=49, name='Betty')

In [10]:
p2_dc = PersonDC(id=4, age='old', name='Jonas') 

As you can see, a Python `dataclass` will not perform validation out-of-the-box.

## Serialization/deserialization

`pydantic` objects can be easily serialized to JSON.

In [11]:
p1.json()

'{"id":1,"age":72,"name":"Maria"}'

The inverse, deserialization from JSON is trivial as well.

In [12]:
json_str = '''
    {
        "id": 2,
        "age": 15,
        "name": "Berthe"
    }
'''

In [13]:
Person.model_validate_json(json_str)

Person(id=2, age=15, name='Berthe')

When the JSON representation doesn't represent a valid `Person`, a `ValidationError` is raised.

In [14]:
json_str = '{"id": 5, "age": "old", "name"="Johnny"}'
try:
    _ = Person.model_validate_json(json_str)
except ValidationError as e:
    print(e, file=sys.stderr)

1 validation error for Person
  Invalid JSON: expected `:` at line 1 column 31 [type=json_invalid, input_value='{"id": 5, "age": "old", "name"="Johnny"}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/json_invalid


## Inheritance

A child is of course a person, but aged younger than 18.  The `Child` class can be derived from the `Person` class, but with a restriction on its age.  In order to enforce this, the attribute should be a `pydantic` `Field` object.  The latter has a `lt` attribute, which stand for 'less than'.

In [15]:
class Child(Person):
    age: int = Field(ge=0, lt=18)

Note that also the niminum age was specified.  Of course, it would have made more sense to define that for `Child`'s parent class, `Person`.

In [16]:
p2 = Child(id=3, age=5, name='Bert')

When you try to create a `Child` object with an inappropriate age, a `ValidationError` is raised.

In [17]:
try:
    _ = Child(id=4, age=20, name='Annie')
except ValidationError as e:
    print(e, file=sys.stderr)

1 validation error for Child
age
  Input should be less than 18 [type=less_than, input_value=20, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/less_than


## Schema

In [18]:
Person.model_json_schema()

{'properties': {'id': {'title': 'Id', 'type': 'integer'},
  'age': {'title': 'Age', 'type': 'integer'},
  'name': {'title': 'Name', 'type': 'string'}},
 'required': ['id', 'age', 'name'],
 'title': 'Person',
 'type': 'object'}

In [19]:
Child.model_json_schema()

{'properties': {'id': {'title': 'Id', 'type': 'integer'},
  'age': {'exclusiveMaximum': 18,
   'minimum': 0,
   'title': 'Age',
   'type': 'integer'},
  'name': {'title': 'Name', 'type': 'string'}},
 'required': ['id', 'age', 'name'],
 'title': 'Child',
 'type': 'object'}

In [52]:
class BetterPerson(BaseModel):
    id: int = Field(ge=0)
    first_name: str = Field(pattern=r'^[A-Za-z- ]+$')
    last_name: str = Field(pattern=r'^[A-Za-z- ]+$')
    age: int

    @field_validator('age')
    @classmethod
    def validate_age(cls, age: int):
        if age < 0 or age > 120:
            raise ValueError('age should be between 0 and 120')
        return age

In [53]:
try:
    _ = BetterPerson(id=0, first_name='A', last_name='Knuth45', age=68)
except ValidationError as e:
    print(e, file=sys.stderr)

1 validation error for BetterPerson
last_name
  String should match pattern '^[A-Za-z- ]+$' [type=string_pattern_mismatch, input_value='Knuth45', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/string_pattern_mismatch


In [54]:
BetterPerson(id=0, first_name='Geert Jan', last_name='Bex', age=68)

BetterPerson(id=0, first_name='Geert Jan', last_name='Bex', age=68)

In [55]:
try:
    _ = BetterPerson(id=7, first_name='Jack', last_name='Ripper', age=150)
except ValidationError as e:
    print(e, file=sys.stderr)

1 validation error for BetterPerson
age
  Value error, age should be between 0 and 120 [type=value_error, input_value=150, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error
